# **Diplomado IA: Aplicaciones de Inteligencia Artificial I - Parte 2**. <br> Práctico 4: Generación de resúmenes
---
---

**Profesores:**
- Vladimir Araujo
- Felipe del Río

**Ayudante:**
- Nissim Ergas
---
---

# **Instrucciones Generales**

El siguiente práctico se debe realizar de forma **individual**. El formato de entregar es el **archivo .ipynb con todas las celdas ejecutadas**. Todas las preguntas deben ser respondida en celdas de texto. No se aceptará el _output_ de una celda de código como respuesta.

**Nombre:** FRANCISCO MENA

**Fecha de entrega: Viernes 8 de enero.**

El siguiente práctico cuanta con 2 secciones donde cada una contendrá 1 o más actividades a realizar. Algunas actividades correspondrán a escribir código y otras a responder preguntas. 

**Importante.** Para facilitar su ejecución, cada sección puede ser ejecutada independientemente.

Se recomienda **fuertemente** revisar las secciones donde se entrega código porque algunas actividades de código pueden reutilizar el mismo código pero con cambios en algunas líneas.



# **Agenda**

>[Diplomado IA: Aplicaciones de Inteligencia Artificial I - Parte 2.  Práctico 4: Generación de resúmenes](#scrollTo=tHopPtVaNF1K)

>[Instrucciones Generales](#scrollTo=uIdAKAdELPSl)

>[Agenda](#scrollTo=kEloa5uXLIPK)

>[Parte I: Generación de Resumenes Extractivos](#scrollTo=Ww8OOQyAnBQj)

>>[Preámbulo](#scrollTo=jdQWS0upPEir)

>>>[Funciones Auxiliares](#scrollTo=UBo9B9xujjlr)

>>[Modelo de resumenes extractivo](#scrollTo=7b6hqPsgPKhM)

>>>[Entrenamiento](#scrollTo=Ge3EMmSDSut6)

>>>[Configuración de nuestra ejecución](#scrollTo=3Cw_GQPmgsul)

>>>[Datos](#scrollTo=dV73kM9nihPx)

>>>[Evaluación](#scrollTo=l5hju-cES9hD)

>>>[Actividad 1](#scrollTo=cZ3WoiKWYMAz)



# Parte I: Generación de Resumenes Extractivos

En esta actividad aplicaremos lo que vimos acerca de métodos para resumir de forma extractiva. Nos basaremos en un modelo que sigue el proceso planteado para modelos extractivos que vimos en clase. Este modelo viene del trabajo [Fine-tune BERT for Extractive Summarization](https://arxiv.org/pdf/1903.10318.pdf), a continuación se detalla un diagrama mostrando la forma en que funciona este modelo. 

<figure>
<center>
<img src='https://docs.google.com/drawings/d/e/2PACX-1vSa16s1rK-JLF_E13C5E4vLe6qRp7azTi-7tLXpUTVisr-OUntDVs-nezE4sWH4QCOLPdwf_JCxvIut/pub?w=765&h=494' height="250" />
<figcaption>Cálculo de precision y recall.</figcaption>
</center>

</figure>

Por el momento no entraremos en detalle de la arquitectura de *transformer*, pues lo verán en el futuro. Pero si les gana la curiosidad, les recomiendo leer el paper que los propuso, ["Attention is all you need"](https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf) en NeurIPS 2017, y lo mismo con el paper de [BERT](https://arxiv.org/pdf/1810.04805.pdf) publicado en ACL 2019.

Este modelo de lenguaje será visto en la próxima sección del diplomado. 


<small>Basado en la [implementación oficial](https://github.com/nlpyang/BertSum) de este trabajo.</small>


## Preámbulo
Primero debemos copiar el dataset y otros archivos que utilizaremos hacia colab. El dataset lo decomprimiremos dentro del directorio data, que recién creamos. También instalaremos algunas de las librerías que ocuparemos durante esta actividad. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

Primero que todo descargaremos las librerías que vamos a utilizar, en este caso debemos modificar la versión de PyTorch que utilizaremos a la `1.1.0`. Esto se debe a que la implementación original utilizó esa versión y así mantenmos mejor compatibilidad.

In [2]:
!wget -nc https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl -q --show-progress
!wget -nc https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl -q --show-progress
!pip uninstall -qy torch torchvision fastai
!pip install -q urllib3==1.25.10 folium==0.2.1 # Nos evitan mensajes de errores, aunq ue no influyen en la ejecución
!pip install -q torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip install -q torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

torch-1.1.0-cp36-cp 100%[===================>] 734.97M  76.6MB/s    in 11s     
torchvision-0.3.0-c 100%[===================>]   2.46M  15.1MB/s    in 0.2s    
     |████████████████████████████████| 133kB 8.9MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 


In [3]:
!pip install -q pytorch_pretrained_bert tensorboardX pyrouge

     |████████████████████████████████| 133kB 8.1MB/s 
     |████████████████████████████████| 317kB 13.5MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 133kB 12.7MB/s 
     |████████████████████████████████| 7.2MB 13.4MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 


In [4]:
!git clone --progress https://github.com/nlpyang/BertSum.git

Cloning into 'BertSum'...
remote: Enumerating objects: 301, done.
remote: Total 301 (delta 0), reused 0 (delta 0), pack-reused 301
Receiving objects: 100% (301/301), 15.03 MiB | 20.42 MiB/s, done.
Resolving deltas: 100% (174/174), done.


Debemos también descargar los archivos del modelo pre-entrenado. 

In [5]:
!wget -nc https://raw.githubusercontent.com/Hernan4444/MyScripts/master/google_drive/google_api.py -q --show-progress

import os
from google_api import download_file_without_authenticate

if not os.path.exists('lab4.zip'): 
    download_file_without_authenticate('18ecT1r9jFGyChATPwTdRgvj9dKctd3sS', 'lab4.zip')

google_api.py       100%[===================>]   5.55K  --.-KB/s    in 0s      


También debemos descargar los datos en donde probaremos nuestros modelos.

In [6]:
!unzip -nq lab4.zip -d .
!mv lab4/activity\ 1/model.pt BertSum/models/model.pt

In [7]:
download_file_without_authenticate('1x0d61LP9UAN389YN00z0Pv-7jQgirVg6', 'bertsum_data.zip')
!unzip -nq bertsum_data.zip -d BertSum/bert_data

In [8]:
os.chdir('/content/BertSum/src/')

Finalmente configuraremos nuestra ejecución.

In [9]:
class Args():
    encoder='classifier'                        # Tipo de encoder a utilizar
    mode='test'                                 # Testearemos el modelo, no lo entrenaremos
    bert_data_path='../bert_data/cnndm'         # Path a los datos
    model_path='../models/'                     # Path a los checkpoints
    result_path='../results/cnndm'              
    temp_dir='../temp'
    bert_config_path='../bert_config_uncased_base.json' # Archivo de configuración de hugginface de BERT 
    batch_size=1000
    use_interval=True
    hidden_size=128
    ff_size=512
    heads=4
    inter_layers=2
    rnn_size=512
    param_init=0
    param_init_glorot=True
    dropout=0.1
    optim='adam'                                # Optimizador a utilizar
    lr=1                                        # Tasa de aprendizaje
    beta1= 0.9
    beta2=0.999
    decay_method=''
    warmup_steps=8000
    max_grad_norm=0
    save_checkpoint_steps=5
    accum_count=1
    world_size=1
    report_every=1
    train_steps=1000
    recall_eval=False
    visible_gpus=-1
    gpu_ranks=[0]
    log_file='../logs/cnndm.log'
    dataset=''
    seed=1666
    test_all=False
    test_from=''
    train_from=''
    report_rouge=False
    block_trigram=True   

### Funciones Auxiliares

Definamos algunas funciones que nos ayudarán a visualizar sus resultados de mejor forma.

In [10]:
import textwrap

# Auxiliary function to help in the displaying of results
def bold(text):
  return '\033[1m' + text + '\033[0m'

def highlight(text):
  return '\033[43m' + text + '\033[0m'

def display_translation(srcs, tgts, preds=None):
    bold_end = bold('<sent_end>')
    preds = [None] * len(srcs) if preds is None else preds
    for idx, (src, tgt, pred) in enumerate(zip(srcs, tgts, preds), start=1):
        print(bold(f'Sample {idx}'))
        print(bold('Source Text:'))
        for sent in src:
            sent = highlight(sent) if (pred is not None) and (sent in pred.split('\n')) else sent
            print(textwrap.fill(sent, width=70), end=f' {bold_end}\n')
        print()

        print(bold('Gold Standard Summarization:'))
        # print(textwrap.fill(tgt, width=70), end=' {bold_end}\n\n')
        for sent in tgt.split('<q>'):
            print(textwrap.fill(sent, width=70), end=f' {bold_end}\n')
        print()

        if pred is not None:
            print(bold('Predicted Summarization:'))
            # for sent in pred:
            # print(textwrap.fill(pred, width=70), end=f' {bold_end}\n\n')
            for sent in pred.split('\n'):
                print(textwrap.fill(sent, width=70), end=f' {bold_end}\n')
            print() 

        print('                 ===========================                  \n')

# Evaluation
def _get_ngrams(n, text):
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set

def _block_tri(c, p):
    tri_c = _get_ngrams(3, c.split())
    for s in p:
        tri_s = _get_ngrams(3, s.split())
        if len(tri_c.intersection(tri_s))>0:
            return True
    return False

## Modelo de resumenes extractivo

En esta sección tomaremos un modelo para generar resumenes extractivos pre-entrenado y veremos como funciona en el dataset de resumenes de noticias CNN/DailyMail.

En esta actividad no entrenaremos un modelo, de todas maneras está disponible el código para que uds entrenen su propio modelo en caso de que así lo deseen.

Partimos importando los paquetes que utilizaremos y utilizando las configuraciones que seteamos previamente.

In [11]:
import numpy as np
import torch
from pytorch_pretrained_bert import BertConfig

from models import data_loader
from models.data_loader import load_dataset
from models.model_builder import Summarizer
from models.trainer import build_trainer

self_trained = False

### Entrenamiento

Para poder entrenar su propio modelo deben descomentar la siguientes **dos celdas** de código. 

Si se fijan bien, estamos configurando nuestro entrenamiento con los argumentos que le pasamos al script al llamar a `python train.py`. En base a estos argumentos pueden modificar el modelo y el entrenamiento mismo, sin embargo es importante que seteen correctamente los siguientes parámetros. Aun que si corren como está por defecto no deberían tener problemas.

- `bert_data_path`: Path a los datos de entrenamiento, estos deben seguir el formato indicado en [esta sección](https://github.com/nlpyang/BertSum/#data-preparation-for-cnndailymail) del repositorio.
- `model_path`: Path a donde serán guardados los checkpoints del entrenamiento.
- `log_file`: Path al log del entrenamiento.
- `batch_size`: El tamaño del batch, si tienen problemas de memoria de la GPU (no deberían en colab) pueden disminuírlo.
- `train_steps`: Número de iteraciones del entrenamiento, el entrnemineto original consta de 50.000 iteraciones, pero pueden probar con menos. Para este caso también pueden setearlo en la variable de entorno `$TRAIN_STEPS`


De todas maneras pueden encontrar más información en el [repositorio del proyecto](https://github.com/nlpyang/BertSum/#model-training).

Es importante notar que el entrenamiento durará un par de horas en caso de que quieran realizarlo.

In [20]:
self_trained = True

In [21]:
%%shell
TRAIN_STEPS=1000 # Entrenamiento corto de prueba, originalmente 50.000
python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path ../bert_data/cnndm \
                -model_path ../models/bert_classifier -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 \
                -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 1000 \
               -decay_method noam -train_steps $TRAIN_STEPS -accum_count 2 -log_file ../logs/bert_classifier \
              -use_interval true -warmup_steps 10000
mv ../models/bert_classifier/model_step_${TRAIN_STEPS}.pt ../models/model-self-trained.pt

[2021-01-03 20:51:26,911 INFO] Device ID 0
[2021-01-03 20:51:26,911 INFO] Device cuda
[2021-01-03 20:51:27,213 INFO] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmpzuqy368v
100% 407873900/407873900 [00:11<00:00, 34880638.90B/s]
[2021-01-03 20:51:39,317 INFO] copying /tmp/tmpzuqy368v to cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2021-01-03 20:51:40,413 INFO] creating metadata file for ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2021-01-03 20:51:40,414 INFO] removing temp file /tmp/tmpzuqy368v
[2021-01-03 20:51:40,465 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca

### Configuración de nuestra ejecución

Para poder configurar correctamente nuestra ejecución utlizaremos los argumentos previamente definidos, para que estos estén ad-hoc a como está dispuesto nuestro entorno de colab. También consideraremos los argumentos que se usaron en el entrenamiento para mantenerlos igual en la creación del modelo.

Estos últimos vienen en el checkpoint que tenemos del entrenamiento, así que lo leemos y actualizamos nuestros argumentos.

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

args = Args()
checkpoint_path = f'{args.model_path}/model.pt' if not self_trained else f'{args.model_path}/model-self-trained.pt'
checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage)

opt = vars(checkpoint['opt'])
for k in opt.keys():
    if (k in model_flags):
        setattr(args, k, opt[k])

### Datos

Para poder evaluar nuestro modelo primero debemos cargar los datos de test. Además podemos aprovechar de ver la forma que tienen estos datos y hacernos una mejor idea de la tarea que está haciendo el modelo.

In [24]:
test_iter = data_loader.Dataloader(args, load_dataset(args, 'test', shuffle=True),
                              args.batch_size, device,
                              shuffle=False, is_test=True)

In [25]:
batch = next(iter(test_iter))
display_translation(batch.src_str, batch.tgt_str)

Sample 1
Source Text:
chinese property conglomerate dalian wanda group have formalised their
purchase of 20 per cent of spanish champions atletico madrid . <sent_end>
the purchase was formalised at an extraordinary general meeting on
tuesday where the legal team representing wanda madrid investment
signed off on the purchase of 726,707 shares for $ 45million ( # 32.8
m ) . <sent_end>
the deal was initially announced in january but two egms were needed
in order to ratify the deal . <sent_end>
wanda group chairman wang jianlin hopes to grow atletico 's brand in
asia <sent_end>
atletico madrid manager diego simeone speaks to the media <sent_end>
wang jianlin , chairman of wanda group , said : ` wanda group is
delighted with the possibility of contributing to the growth of
atletico madrid and the development of its brand in asia , as well as
being able to rely on their extraordinary experience in the training
area , which no doubt will be very useful in the growth of base
football in china

### Evaluación

Evaluaremos nuestro modelo cualitativamente. Para esto primero debemos crear nuestro modelo y cargar los pesos que teníamos pre-entrenados.

Utilizaremos la clase `Summarizer` que es a su vez un `nn.Module` de PyTorch. Esta clase integra los distintos módulos que se plantearon anteriormente para crear un resumen en base a los datos entregados. 

Finalmente cargamos los pesos utilizando la función `Summarizer.load_cp`, que recibe el mismo `checkpoint` que leímos anteriormente.

In [26]:
config = BertConfig.from_json_file(args.bert_config_path) # configuracion para la librería HuggingFace Transformers
model = Summarizer(args, device, load_pretrained_bert=False, bert_config=config)
model.load_cp(checkpoint)

Ya creado nuestro modelo, podemos ver algunos ejemplos de las predicciones que este hace. Aquí:

- **Source Text:** Corresponde al documento que será resumido. En este caso al tratarse de un modelo extractivo destacamos aquí mismo las frases seleccionadas.
- **Gold Standard Summarization:** Corresponde al resumen de referencia creado para este text, se usa a modo de "ground truth".
- **Predicted Summarization:** Corresponde al resumen generado por nuestro modelo. En este caso al ser un modelo extractivo, es una selección de oraciones del texto original.

In [27]:
model.eval()
with torch.no_grad():
    batch = next(iter(test_iter))

    src = batch.src
    labels = batch.labels
    segs = batch.segs
    clss = batch.clss
    mask = batch.mask
    mask_cls = batch.mask_cls

    gold = []
    pred = []

    sent_scores, mask = model(src, segs, clss, mask, mask_cls)

    sent_scores = sent_scores + mask.float()
    sent_scores = sent_scores.cpu().data.numpy()
    selected_ids = np.argsort(-sent_scores, 1)

    for i, idx in enumerate(selected_ids):
        _pred = []
        if len(batch.src_str[i]) == 0:
            continue
        for j in selected_ids[i][:len(batch.src_str[i])]:
            if j >= len(batch.src_str[i]):
                continue
            candidate = batch.src_str[i][j].strip()
            if args.block_trigram:
                if(not _block_tri(candidate, _pred)):
                    _pred.append(candidate)
            else:
                _pred.append(candidate)

            if ((not args.recall_eval) and len(_pred) == 3):
                break

        _pred = '\n'.join(_pred)
        if (args.recall_eval):
            _pred = ' '.join(_pred.split()[:len(batch.tgt_str[i].split())])

        pred.append(_pred)
        gold.append(batch.tgt_str[i])

display_translation(batch.src_str, gold, pred)

Sample 1
Source Text:
australian prime minister tony abbott thrilled players at a sydney
australian rules football club function by skolling a beer . <sent_end>
mr abbott was asked to have a drink by university of technology sydney
bats coach simon carradous , and the prime minister happily obliged . <sent_end>
it appeared to take the prime minister about six seconds to down
the schooner , as players cheered and chanted ` skol ' and ` tony '
. <sent_end>
australian prime minister tony abbott thrilled players at a sydney
australian rules football club function by skolling a beer <sent_end>
it appeared to take the prime minister about six seconds to down the
schooner , as players cheered and chanted ` skol ' and ` tony ' <sent_end>
mr abbott also gave a speech to the players , according to
australian women 's weekly . <sent_end>
` he grabs the microphone and gives a speech , which went along the
lines of : ` well is n't this a real treat , i 've been a rugby man
all my life but now i fin

### Actividad 1

Ejecute el notebook completo incluyendo un entrenamiento de tan solo 1.000 iteraciones.

**Ayuda:** Debe seguir las instrucciones detalladas en la sección [Entrenamiento](#scrollTo=Ge3EMmSDSut6) y continuar la ejecución normalmente.

**Importante:** Para la entrega de esta actividad el notebook debe tener incluída la ejecución del entrenamiento para obtener puntaje.